## Proposta

Encontrar os valores dos campos de controle, $f_x$, $f_y$ e $f_z$. Os valores de controle podem ser definidos como: $$f_{x,y,z}=-a_{x,y,z}S(A_{x,y,z},t) - h_{x,y,z}\frac{S(B,t)}{S(A_{x,y,z},t)}$$

$S$ é a representação do controle de Lyapunov já que este consiste nas componentes que formam a derivada de $V(t)$, cujo o propósito, em Lyapunov, é ser menor que zero enquanto $V(t)$ deve ser maior que zero, em qualquer tempo. $S$ pode ser definido como: $$S(X,t)=(-\frac{1}{2}U_t^{\dagger}X^{\dagger}U_fW^{\dagger}-\frac{1}{2}W^{\dagger}U_t^{\dagger}X^{\dagger}U_f+2U_t^{\dagger}X^{\dagger}U_f)L$$

Para começar os trabalhos, iniciamos em $t=0$ pois como $U$ é a evolução temporal unitária, $U_0 = diag(1,1,1)$, ademais, $X \in \{A_x, A_y, A_z, B\}$, $U_f=\left(\begin{array}{cc} 
0 & 0 & 1\\
0 & -1 & 0\\
1 & 0 & 0\\
\end{array}\right)$, $h_x+h_y+h_z = 1$. O $A's$ e $B$ são matrizes que foram obtidas por meio da transformação adjunta em $SO(3)$ da equação mestra de Lindblad, por fim, $W_t = U_f^{\dagger}U_t$ e por consequência $L_t = W_t - I -\frac{1}{2}(W_t-I)^2$.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
import random

In [2]:
def lyap(Ui, Uf, X):
    first=-1/2*np.matmul(np.matmul(np.matmul(Ui.getH(),X.getH()),Uf),np.matmul(Uf.getH(), Ui).getH())
    second=-1/2*np.matmul(np.matmul(Uf.getH(), Ui).getH(), np.matmul(np.matmul(Ui.getH(),X.getH()),Uf))
    third=2*np.matmul(np.matmul(Ui.getH(),X.getH()),Uf)
    L=np.subtract(np.matmul(Uf.getH(), Ui), np.identity(3))-1/2*np.subtract(np.matmul(Uf.getH(), Ui), np.identity(3))**2
    ##return 2*np.trace(first+second+third)
    print(f"first={first}")
    print(f"second={second}")
    print(f"third={third}")
    print(f"L={L}")
    return (first+second+third)*L

def control(Ui, Uf, a, h, A, B):
    print(f"A={A}")
    SA=lyap(Ui, Uf, A)
    SB=lyap(Ui, Uf, B)
    print(f"SA={SA}")
    print(f"SB={SB}")
    return -a*SA-h*(SB/SA)

# U' = (A(t)+B)U
def dUdt(u, Ui, Uf):
    Ax = np.matrix('0 0 0; 0 0 -1; 0 1 0')
    Ay = np.matrix('0 0 1; 0 0 0; -1 0 0')
    Az = np.matrix('0 -1 0; 1 0 0; 0 0 0')
    B = 0.01*np.matrix('-1 0 0; 0 -1 0; 0 0 -2')
    F = np.dot(control(Ui, Uf, u[0], u[3], Ax, B), Ax)+np.dot(control(Ui, Uf, u[1], u[4], Ay, B), Ay)+np.dot(control(Ui, Uf, u[2], u[5], Az, B),Az)+B
    return np.matmul(F, Ui)

# Runge-Kutta de quarta ordem

def runge_kutta(u, Ui, Uf, h):
    
    k1 = dUdt(u, Ui, Uf)
    k2 = dUdt(u, Ui + np.dot((0.5*h), k1), Uf)
    k3 = dUdt(u, Ui + np.dot((0.5*h), k2), Uf)
    k4 = dUdt(u, Ui + np.dot(h, k3), Uf)
    
    A = np.dot(2,k3) + k4
    B = np.dot(2,k2) + k1
    C = A+B

    res = Ui + np.dot((h / 6.0),(C))
        
    return res

In [3]:
def func_objetivo(u, Ui, Uf, h):
    return np.linalg.norm(np.subtract(Uf, runge_kutta(u, Ui, Uf, h)))

In [4]:
def const(u): #h_x+h_y+h_z=1
    return u[3] + u[4] + u[5] - 1

def main():
    
    # Instância
    cons = {'type':'eq', 'fun': const}
    u = []
    # a's
    for i in range(0,3):
        u.append(random.uniform(-5,5))
    
    #h's
    u.append(0)
    u.append(0)
    u.append(1)

    h = 0.001 #Runge-kutta

    t = 0
    steps = 100

    Ui = np.matrix('1 0 0; 0 1 0; 0 0 1')
    Uf = np.matrix('0 0 1; 0 -1 0; 1 0 0')

    while t < 100:

        print(f'U_{t}: {Ui}')
        op = optimize.minimize(func_objetivo, u, args=(Ui, Uf, h), constraints=cons, method='SLSQP')
        print(f'The optimization result is: {op.x}')
        u = op.x
        Ui = runge_kutta(u, Ui, Uf, h)

        t = t + 1

    

In [5]:
main()

U_0: [[1 0 0]
 [0 1 0]
 [0 0 1]]
A=[[ 0  0  0]
 [ 0  0 -1]
 [ 0  1  0]]
first=[[ 0.   0.   0. ]
 [ 0.   0.  -0.5]
 [ 0.   0.5  0. ]]
second=[[ 0.  -0.5  0. ]
 [ 0.5  0.   0. ]
 [ 0.   0.   0. ]]
third=[[0 0 0]
 [2 0 0]
 [0 2 0]]
L=[[-2.  0.  2.]
 [ 0. -4.  0.]
 [ 2.  0. -2.]]
first=[[0.005 0.    0.   ]
 [0.    0.005 0.   ]
 [0.    0.    0.01 ]]
second=[[0.01  0.    0.   ]
 [0.    0.005 0.   ]
 [0.    0.    0.005]]
third=[[ 0.    0.   -0.02]
 [ 0.    0.02  0.  ]
 [-0.04  0.    0.  ]]
L=[[-2.  0.  2.]
 [ 0. -4.  0.]
 [ 2.  0. -2.]]
SA=[[  0.   2.   0.]
 [ -6.   0.   6.]
 [  0. -10.   0.]]
SB=[[-0.07  0.    0.07]
 [ 0.   -0.12  0.  ]
 [ 0.11  0.   -0.11]]
A=[[ 0  0  1]
 [ 0  0  0]
 [-1  0  0]]
first=[[ 0.   0.   0.5]
 [ 0.   0.   0. ]
 [-0.5  0.   0. ]]
second=[[ 0.   0.  -0.5]
 [ 0.   0.   0. ]
 [ 0.5  0.   0. ]]
third=[[-2  0  0]
 [ 0  0  0]
 [ 0  0  2]]
L=[[-2.  0.  2.]
 [ 0. -4.  0.]
 [ 2.  0. -2.]]
first=[[0.005 0.    0.   ]
 [0.    0.005 0.   ]
 [0.    0.    0.01 ]]
second=[[0.01  0

/tmp/ipykernel_4862/2074920608.py:19: RuntimeWarning: divide by zero encountered in divide
  return -a*SA-h*(SB/SA)
/tmp/ipykernel_4862/2074920608.py:19: RuntimeWarning: invalid value encountered in multiply
  return -a*SA-h*(SB/SA)
/tmp/ipykernel_4862/2074920608.py:19: RuntimeWarning: invalid value encountered in divide
  return -a*SA-h*(SB/SA)


first=[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
second=[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
third=[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
L=[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
first=[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
second=[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
third=[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
L=[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
SA=[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
SB=[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
A=[[ 0  0  1]
 [ 0  0  0]
 [-1  0  0]]
first=[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
second=[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
third=[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
L=[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
first=[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
second=[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
third=[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
L=[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]
SA=[[nan nan nan]
 [nan nan nan]
 [nan nan 

KeyboardInterrupt: 